In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.request

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings

warnings.filterwarnings(action='ignore')

# 함수

In [ ]:
def load_table(url, summary_name):
  html = urllib.request.urlopen(url).read()
  soup = BeautifulSoup(html)

  table = soup.find("table", {"summary":f"{summary_name}"} )

  return table

# summary_name - 기업개요, 공모정보, 공모청약일정

In [ ]:
def load_overview(table, read_data=True):
  data = []

  #html = urllib.request.urlopen(url).read()
  #soup = BeautifulSoup(html)

  #table = soup.find("table", {"summary":"기업개요"} )
  get_data = table.find_all('td')

  for i in range(len(get_data)):
    if read_data:
      if i % 2 == 1:
        temp = get_data[i].get_text().replace('\xa0', '').strip()
        data.append(temp)
    else:
      if i % 2 == 0:
        temp = get_data[i].get_text().strip()
        data.append(temp)

  return data

In [ ]:
def load_information(table, read_data=True):
  data = []

  #html = urllib.request.urlopen(url).read()
  #soup = BeautifulSoup(html)

  #table = soup.find("table", {"summary":"공모정보"} )
  get_data = table.find_all('td')

  for i in range(len(get_data)-1):
    if read_data:
      if i % 2 == 1:
        temp = get_data[i].get_text().replace('\xa0', '').strip()
        data.append(temp)
    else:
      if i % 2 == 0:
        temp = get_data[i].get_text().strip()
        data.append(temp)

  # get_manager = get_data[-1].get_text().split('\xa0\xa0/\xa0\xa0')[i].split(':')

  for i in range(2):
    if read_data:
      data.append(get_data[-1].get_text().split('\xa0\xa0/\xa0\xa0')[i].split(':')[1].strip())
    else:
      data.append(get_data[-1].get_text().split('\xa0\xa0/\xa0\xa0')[i].split(':')[0].strip())

  return data

In [ ]:
def load_schedule(table, read_data=True):
  data = []

  #html = urllib.request.urlopen(url).read()
  #soup = BeautifulSoup(html)

  #table = soup.find("table", {"summary":"공모청약일정"} )
  get_data = table.find_all('td')

  if read_data:
    a = get_data[4].get_text().strip().replace('\xa0', '') # 공모청약일
    b = get_data[26].get_text().strip() # 우리사주조합
    c = get_data[27].get_text().strip().split(':')[1].strip() # 우리사주조합 증거금률
    d = get_data[29].get_text().strip().replace('\xa0', '') # 기관투자자
    e = get_data[30].get_text().strip().split(':')[1].strip() # 기관투자자 최고한도
    f = get_data[33].get_text().strip().split(':')[1].strip() # 일반청약자 증거금률
    g = get_data[45].get_text().strip() # 기관경쟁률
    h = get_data[47].get_text().strip() # 의무보유확약
    data = [a,b,c,d,e,f,g,h]
  else:
    a = get_data[3].get_text().strip()
    b = get_data[25].get_text().strip() # 우리사주조합
    c = '우리사주조합 ' + get_data[27].get_text().strip().split(':')[0].strip() # 우리사주조합 증거금률
    d = get_data[28].get_text().strip() # 기관투자자
    e = '기관투자자 ' + get_data[30].get_text().strip().split(':')[0].strip() # 기관투자자 최고한도
    f = '일반투자자 ' + get_data[33].get_text().strip().split(':')[0].strip() # 일반청약자 증거금률
    g = get_data[44].get_text().strip() # 기관경쟁률
    h = get_data[46].get_text().strip() # 의무보유확약
    data = [a,b,c,d,e,f,g,h]

  return data

In [ ]:
def get_full_dataset(url):
  table_overview = load_table(url, '기업개요')
  table_information = load_table(url, '공모정보')
  table_schedule = load_table(url, '공모청약일정')

  data_1 = load_overview(table_overview)
  data_2 = load_information(table_information)
  data_3 = load_schedule(table_schedule)

  data_1.extend(data_2)
  data_1.extend(data_3)

  return data_1

In [ ]:
def get_columns(url):
  table_overview = load_table(url, '기업개요')
  table_information = load_table(url, '공모정보')
  table_schedule = load_table(url, '공모청약일정')

  column_1 = load_overview(table_overview, False)
  column_2 = load_information(table_information, False)
  column_3 = load_schedule(table_schedule, False)

  column_1.extend(column_2)
  column_1.extend(column_3)

  return column_1

# 실행

In [ ]:
url_base = 'http://www.38.co.kr/html/fund/index.htm?o=k&page='

url_list=[]
for i in range(1, 51):
  url = url_base + str(i)
  url_list.append(url)

In [ ]:
link_list = []

for url in url_list:
  html = urllib.request.urlopen(url).read()
  soup = BeautifulSoup(html)

  table = soup.find("table", {"summary":"공모주 청약일정"} )
  line = table.find_all('td', {'height':30})

  for link in line:
    link = link.find('a')
    link = link.attrs['href']
    link = 'http://www.38.co.kr' + link
    link_list.append(link)

In [ ]:
data_list = []

for i in link_list:
  try:
    temp = get_full_dataset(i)
    data_list.append(temp)
  except:
    print(i)

columns = get_columns(link_list[0])

# http://www.38.co.kr/html/fund/?o=v&no=1862&l=&page=3
# http://www.38.co.kr/html/fund/?o=v&no=1437&l=&page=15
# http://www.38.co.kr/html/fund/?o=v&no=1263&l=&page=21

http://www.38.co.kr/html/fund/?o=v&no=1862&l=&page=3
http://www.38.co.kr/html/fund/?o=v&no=1437&l=&page=15
http://www.38.co.kr/html/fund/?o=v&no=1263&l=&page=21


In [ ]:
data = pd.DataFrame(data_list, columns=columns)

In [ ]:
data.to_csv('상장주정보.csv')